In [ ]:
#imports
import torch #entire library
import torch.nn as nn #neural network
import torch.optim as optim #optimizaztoin alg
import torch.nn.functional as F #activation functions
from torch.utils.data import DataLoader #create with batches and work with data
import torchvision.datasets as datasets
import torchvision.transforms as transforms



In [ ]:
#Create fully connected network

class NN(nn.Module):
  def __init__(self, input_size, num_classes): #input_size = 28x28 images = 784 nodes
    super(NN,self).__init__()
    self.fc1 = nn.Linear(input_size, 50) #First Layer
    self.fc2 = nn.Linear(50, num_classes) #hidden layer w 50 nodes

  def forward(self, x):
    x= F.relu(self.fc1(x)) #relu is linearity functions
    x = self.fc2(x)
    return x

#generate random model to test network
model = NN(784, 10) # 10 = num digits, 
x = torch.randn(64, 784) # 64 = num images/exaples to run (batch size)
print(model(x).shape) # want shape of (64, 10), want to return 10 val for each example

#WHY IS (64,10) THE OUTPUT??


torch.Size([64, 10])


In [ ]:
#Create a simple CNN
class CNN(nn.Module):
  def __init__(self, in_channels = 1 , num_classes = 10): #in_channels = rgb
    super(CNN, self).__init__()
    #This is called a "same" convolutions, dont lose any dimensions in filter
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2)) #will cut output to half size
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)  #what is fc1??
  

    return x

model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)
exit()


torch.Size([64, 10])


In [ ]:
#Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#HyperParameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [ ]:
#load data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

#where does the data get downloaded to?, how does it know we wanted these specific images?


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [ ]:
#Initialize network
model= NN(input_size=input_size, num_classes=num_classes).to(device)


In [ ]:
#Loss and optimizer
criterion = nn.CrossEntropyLoss() #Loss Function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
#Train Network

for epoch in range(num_epochs):   #epoch means model seen all images in dataset
  for batch_idx, (data, targets) in enumerate(train_loader): #batch_idx gives us the index, provided by enumerate
              #data = image, target = label
    
    #get data to device
    data = data.to(device=device)
    targets = targets.to(device=device)

    #print(data.shape) #num_images = 64, num_channels=1(rgb colors), image 28x28

    #get correct shape
    data = data.reshape(data.shape[0], -1) #flatten last 3 columns into 1
    #print(data.shape) #64 images, 784 dimensions

    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad() #set gradient to zero so doesnt store previous foward, learningRate = gradientLoss X lr
    loss.backward()

    #gradient descent or adam step
    optimizer.step()




In [ ]:
#Check accuracy on test to see how good our model is
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("CHecking with training data")
  else:
    print("Checking with test data")
  num_correct = 0
  num_samples = 0
  model.eval() #this makes it evaluation mode?

  with torch.no_grad(): #dont compute gradients
    for x, y in loader:
      x=x.to(device=device)
      y=y.to(device=device)
      x=x.reshape(x.shape[0], -1)

      scores = model(x)  #64x10
      _, predictions = scores.max(1) #max of the second dimension (64x10), 
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0) #first dimension, so 64

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


CHecking with training data
Got 55939 / 60000 with accuracy 93.23
Checking with test data
Got 9334 / 10000 with accuracy 93.34
